In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random 
import pickle
import argparse

device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

# hyper params
batch_size = 32 
block_size = 64 # sequence length
max_iters = 300
learning_rate = 3e-4
eval_iters = 25
n_embd = 384
n_layer = 1 # number of decoded blocks
n_head = 1
dropout = 0.2


mps


In [2]:
# opening the text 
chars = ""
with open('openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))

vocab_size = len(chars)

#  wizard of oz text
# with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
#     text = f.read()
    
# chars = sorted(set(text))
# vocab_size = len(chars)


    
# print(chars)

In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# entire wizard of oz text as data
data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [4]:
# getting chunks from the split
def get_random_chunk(split):
    filename = 'openwebtext/train_text.txt' if split == 'train' else 'openwebtext/valid_text.txt'
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # calculate the file size and randomise the starting position 
            file_size = len(mm)
            start_pos = random.randint(0, (file_size )- block_size*batch_size)
            
            # now look for the random block and read all text int hat block 
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)
            
            # decode into a string, ignore errors, ignore invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # train test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    
    return data


# ----------------------------------------------------------------------------------------------------------------------------
#  splitting for the wizard of oz text 

# n = int(0.8*len(data))
# train_data = data[:n]
# val_data = data[n:]

# ----------------------------------------------------------------------------------------------------------------------------

def get_batch(split):
    data = get_random_chunk(split)

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# x, y = get_batch('train')
# print('inputs:')
# #print(x.shape)
# print(x)
# print('targets:')
# print(y)

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        # expected values 
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # register the no look-ahead masking in the model state (just to reduce the training times)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
      
    # attention heads
    def forward(self, x):
        # input of size (batch, time-stamp, channels)
        # output of (batch, time-stamp, head_size)
        B,T,C = x.shape
        k = self.key(x) # (B, T, HS)
        q = self.query(x)
        # now we compute the attention scores ('affinities')
        # weighted
        wei = q @ k.transpose(-2,-1) * k.shape[-1] **0.5  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # masked fill prevents lookahead
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        # Softmax to sharpen some values
        wei = F.softmax(wei, dim =-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B, T, HS)
        # finally out weighted value is multiplied by our weights
        out = wei @ v # ()
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size):
        
        super().__init__()
        # create number of heads running in parallel and place them into the Module list 
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # projection: projection of the 
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        # drop 20% of the networks neurons
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)# (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out
        
class FeedForward(nn.Module):
    
    """ a simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear( 4 * n_embd,n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    
    """ Transformer block: communication followed by computation """
    
    def __init__(self, n_embd, n_head):
        # n_embd is embedding dimensionality, n_head is the number of heads we'd like
        super().__init__()
        # the number of features each head will be capturing in the MultiHeadAttention 
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln1(x + y)
        return x

# Logits typically refer to the raw scores or predictions generated by your model before applying softmax
# Targets represent the ground truth labels for your examples. These are the correct classes that you want your model to predict.

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Initialise embedding tables
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) 
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # creating the blocks (decoding blocks) sequentially
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for i in range(n_layer)])
        # Normalization using Layer Normalization
        self.ln_f = nn.LayerNorm(n_embd) 
        # The Linear Transformation defined with input size n_embd and output size vocab_size
        self.lm_head = nn.Linear(n_embd, vocab_size) 
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.0)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init
    
    # the forward propagation function including calculating the Loss 
    def forward(self, index, targets=None):
        # initializing the weights
        B, T = index.shape
        
        # index and targets (B, T) are both tensors of integers 
        tok_emd = self.token_embedding_table(index) # (B,T,C)
        pos_emd = self.position_embedding_table(torch.arange(T, device=device))# (T,C)
        x = tok_emd + pos_emd # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            # B representing batch size, T representing the sequence length (block_size), and c representing the original embedding dimensionality 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            
            # get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] 
            # because we only selected the last step in each batch, logits now only has two dimensions. 
            # Of the original B, T, C as T is the last one chose, we are left with just B, C. A (2,2) matrix
            # use soft max to find the probabilities 
            probs = F.softmax(logits, dim=-1 )# (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # finally append the sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) # (B, T+1)
            
        return index  
    
model = GPTLanguageModel(vocab_size)

# print('Loading model params.....')
# with open('BigramLM-GPT-model-01.pkl', 'rb') as f:
#     model=pickle.load(f)
# print('Loading complete!')

m = model.to(device)


# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [7]:
#  creating the PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
    # sample batch of data, xb is the predictions, yb is the targets, during the initiation this is set to None
    xb, yb =get_batch('train')
    
    # evaluate the loss using standard training loop architecture for basic models
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
#  A scalar representation of how the model is doing, compared to the true values calculated during 'Forward Propagation'
# A lowe value indicates an improvement of the Bigram Language Model
print(loss.item())

with open('BigramLM-GPT-model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model has been saved!')

0
step: 0, train loss: 10.511, val loss: 10.481
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
step: 25, train loss: 4.006, val loss: 4.121
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
step: 50, train loss: 2.713, val loss: 2.775
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
step: 75, train loss: 2.472, val loss: 2.619
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
step: 100, train loss: 2.593, val loss: 2.591
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
step: 125, train loss: 2.519, val loss: 2.507
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
step: 150, train loss: 2.403, val loss: 2.293
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
step: 175, train loss: 2.603, val loss: 2.468
176
177
178
179
180
181
182
183
184
185
1

In [ ]:
context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)